<a href="https://colab.research.google.com/github/getaccept/notebooks/blob/master/API_copy_document.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Copy document using GetAccept *API*

Functions to copy document between entities and to create fillable PDF files from GetAccept templates or documents

In [0]:
# import dependencies
import requests
import os
import io
from google.colab import files

In [0]:
#@title Enter GetAccept login information
#@markdown Remember to keep your credentials safe
EMAIL = "" #@param {type:"string"}
PASSWORD = "" #@param {type:"string"}
BASEURL = "https://api.getaccept.com/v1" #@param {type:"string"}
ACTION = "Copy to other entity" #@param ["Copy to other entity", "Create fillable PDF"]
SOURCE_ENTITY = "" #@param {type: "string"}
DESTINATION_ENTITY = "" #@param {type: "string"}
#@markdown Find source document/template id by going to the document in GetAccept \
#@markdown and copy the id from the URL in the browser, eg. abc123de
DOCUMENT_ID = "" #@param {type: "string"}

In [0]:
# Make auth request and store auth header
if EMAIL and PASSWORD:
  payload = { "email": EMAIL, "password": PASSWORD, "entity_id": SOURCE_ENTITY }
  response = requests.post(BASEURL+"/auth", json=payload)
  data = response.json()
  if "access_token" in data:
    source_auth_headers = { "Authorization": "bearer " + data["access_token"]}
  else:
    raise TypeError(data["errors"], "Please check your credentials")
  # Check login and list entities
  response = requests.get(BASEURL+"/users/me", headers=source_auth_headers)
  user_data = response.json()
  print("Logged in as " + user_data["user"]["first_name"] + " on entity " + user_data["user"]["entity_name"])
  if len(user_data["entities"]) > 1:
    print("Available entities:\r\n-", '\r\n- '.join(map(lambda x: x["name"]+": \t"+x["id"], user_data["entities"])))


In [0]:
# If destination is another entity, also auth this
if DESTINATION_ENTITY != "":
  response = requests.get(BASEURL+"/refresh/"+DESTINATION_ENTITY, headers=source_auth_headers)
  data = response.json()
  if "access_token" in data:
    destination_auth_headers = { "Authorization": "bearer " + data["access_token"]}
    print("Authenticated to destination entity " + DESTINATION_ENTITY)
  else:
    raise TypeError(data["errors"], "Please check your DESTINATION_ENTITY parameter")

In [0]:
if DOCUMENT_ID == "":
  # Get selected document or template information
  response = requests.get(BASEURL+"/documents?sort_by=created&sort_order=desc&limit=1",headers=source_auth_headers)
  document_data = response.json()
  document_data
  document_data = document_data[0]
  DOCUMENT_ID = document_data["id"]
else:
  # List latest created documents
  response = requests.get(BASEURL+"/documents/"+DOCUMENT_ID, headers=source_auth_headers)
  document_data = response.json()

if "name" in document_data:
  print("Processing document \"%s\"" % document_data["name"])
else:
  raise TypeError(document_data["errors"], "Please check your DOCUMENT_ID parameter")

In [0]:
if document_data["field_count"] > 0:
  # Get fields for document
  response = requests.get(BASEURL+"/documents/"+DOCUMENT_ID+"/fields",headers=source_auth_headers)
  field_data = response.json()
  print("Fields: " + ", ".join(map(lambda x:x["field_type"]+": "+x["field_value"]+"  "+x["field_label"], field_data["fields"])))

In [0]:
if ACTION == "Copy to other entity":
  # Copy document/template to DESTINATION_ENTITY
  response = requests.get(BASEURL+"/documents/"+DOCUMENT_ID+"/download", headers=source_auth_headers)
  download_data = response.json()
  if "document_url" in download_data:
    pdf_url = download_data["document_url"]
    print("Have document download link")
  else:
    raise TypeError(download_data["error"], "Could not find document")

In [0]:
if ACTION == "Copy to other entity":
  # Insert new document at destination entity
  payload = document_data
  for e in ["id","email_send_template_id","external_id","external_client_id","unique_id","send_date","sign_date","sender_id","sender_email","parent_id","expiration_date","video_id","thumb_url","preview_url","download_url"]: 
    payload.pop(e, None)
  for index,recipient in enumerate(payload["recipients"]):
    if recipient["email"] == "" and recipient["mobile"] == "":
      payload["recipients"].pop(index)
  payload["sender_id"] = user_data["user"]["id"]
  payload["file_url"] = pdf_url
  #if payload["type"] == "template":
  #  payload["type"] = "sales"

  print("Copying document... this might take up to a minute")
  response = requests.post(BASEURL+"/documents", json=payload, headers=destination_auth_headers)
  new_document_data = response.json()
  if "id" in new_document_data:
    print("Successfully copied document " + new_document_data["id"])
  else:
    raise TypeError(new_document_data, "Something went wrong")


In [0]:
if ACTION == "Copy to other entity":
  # Get original document pages
  response = requests.get(BASEURL+"/documents/"+DOCUMENT_ID+"/pages", headers=source_auth_headers)
  page_data = response.json()
  document_page_num = {}
  for page in page_data["pages"]:
    document_page_num[page["page_id"]] = page["page_num"]
  
  print("pages: " + ", ".join(map(lambda x : str(x["page_num"]) + ": " + x["page_id"], page_data["pages"])))
  # print(document_page_num)
  

In [0]:
if ACTION == "Copy to other entity" and "id" in new_document_data and document_data["field_count"] > 0:
  # Copy original fields to the new document
  field_copy_counter = 0
  for field in field_data["fields"]:
    payload = field.copy()
    payload["page_num"] = document_page_num[payload["page_id"]]
    for e in ["page_id","field_id"]: 
      payload.pop(e, None)
    if payload["field_options"]:
      payload["field_options"] = ','.join(map(lambda x : x["value"], payload["field_options"]))
    response = requests.post(BASEURL+"/documents/"+new_document_data["id"]+"/fields", json=payload, headers=destination_auth_headers)
    new_field_data = response.json()
    if "field_id" in new_field_data:
      field_copy_counter = field_copy_counter + 1
    else:
      raise TypeError(new_field_data, "Something went wrong")
  
  print("Copied " + str(field_copy_counter) + " fields to the document/template")

In [0]:
# Download document as PDF
if ACTION == "Create fillable PDF":
  response = requests.get(BASEURL+"/documents/"+DOCUMENT_ID+"/download?direct=true",headers=source_auth_headers)
  download_data = response.content
  pdf_file = DOCUMENT_ID+".pdf"
  with open(pdf_file, "wb") as file:
    file.write(download_data)
  print("Successfully downloaded file to \"%s\"" % (os.getcwd()+"/"+pdf_file))
  # files.download(pdf_file) 

In [0]:
if ACTION == "Create fillable PDF":
  # install dependencies for fillable PDF
  !pip install PyMuPDF
  import fitz

In [0]:
if ACTION == "Create fillable PDF":
  # save down to fillable PDF
  doc = fitz.open(pdf_file)           # or new: fitz.open(), followed by insertPage()
  page = doc[0]                         # choose some page
  page_ratio = page.rect.width/900

  for field in field_data["fields"]:
    field_type = None
    field_flags = None
    choice_values = None
    fill_color = None
    border_style = "S"
    border_width = 0

    field_left = page_ratio*field["field_left"]
    field_top = page_ratio*field["field_top"]
    field_width = page_ratio*field["field_width"]
    field_height = page_ratio*field["field_height"]

    if field["field_type"] == "text":
      field_type = fitz.PDF_WIDGET_TYPE_TEXT
      field_name = field["field_label"]
      field_value = field["field_value"]
      fill_color =  (0.9, 0.9, 0.9)
      border_width = .5
    elif field["field_type"] == "merge":
      field_type = fitz.PDF_WIDGET_TYPE_TEXT
      field_flags = fitz.PDF_FIELD_IS_READ_ONLY
      field_name = field["field_value"]
      field_value = field["field_value"]
    elif field["field_type"] == "signature":
      field_type = fitz.PDF_WIDGET_TYPE_SIGNATURE
      field_flags = None
      field_name = field["field_value"]
      field_value = None
    elif field["field_type"] == "dropdown":
      field_type = fitz.PDF_WIDGET_TYPE_LISTBOX
      field_name = field["field_label"]
      field_value = field["field_value"]
      choice_values = "1,2,3,4" #field["field_options"]

    if field["is_required"]:
      field_flags = fitz.PDF_FIELD_IS_REQUIRED

    if field_type and field_name:
      widget = fitz.Widget() 
      widget.rect = fitz.Rect(field_left, field_top, field_left+field_width, field_top+field_height)
      widget.field_type = field_type
      widget.field_name = field_name
      widget.field_value = field_value
      widget.choice_values = choice_values
      widget.field_flags = field_flags
      widget.text_fontsize = field_height*0.7
      widget.border_style = border_style
      widget.border_color = [0.5, 0.5, 0.5]
      widget.border_width = border_width
      widget.fill_color = fill_color
      annot = page.addWidget(widget)         # add the widget

  # print("unused rectangle height: %g" % rc)          # just demo (should display "44.2")

  doc.save("test.pdf")   # update file with saveIncr. Save to new instead by doc.save("new.pdf",...) 
  files.download("test.pdf") 
  # files.download(pdf_file) 